# To Read
El notebook de este TFM posee la siguiente estructura, detallada a continuación para su comprensión:
- Import Library. Importaciones de Librerías
- Load Data (XML) function. Contiene las funciones necesarias para la lectura del fichero fuente XML de Orphadata, y, a partir de ahí, poder trabajar con los datos.
- Exploratory Data Analysis (EDA) & Feature Engineering functions. Funciones comunes para los dos experimentos, asociadas a la Exploración de los datos, limpieza y preparación de los mismos.
- Test/Experiment: Recomendation functions. Funciones asociadas al Experimento 2, en el que se trabaja con un Sistema Recomendador (planteando 2 alternativas para comparación). Contiene:  ...... bla bla bla bla bala
- Notebook Main: Common Tasks: Es el MAIN principal del notebook. Contiene las llamadas a las diferentes funciones definidas en el notebook para la obtención de los resultados. 



# Import Library
Se importan todas las librerías necesarias para trabajar en el presente Notebook para los dos experimentos que contiene

In [44]:
import xmltodict
import pandas as pd
import numpy as np
import sqlite3
import random
import sklearn
from sklearn.neighbors import NearestNeighbors

# Load Data (XML) function

load_from_xml: Función que extrae en un dataframe los datos de Enfermedades-Síntomas-Frecuencias del XML de Orphadata  (almacenado en una ruta concreta). Es llamada únicamente al principio para trabajar, pues luego se continua trabajando con dataframes y csvs

In [3]:
def load_from_xml (path):
    xml=open(path, encoding='ISO-8859-1')
    xmldict = xmltodict.parse(xml.read())
    df_enfermedades = pd.DataFrame()
    lista_enfer_sinto_prob=[]
    first_tree=xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]
    id=0
    for nodo in xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]:
     
        enfermedad=_clean_name(nodo["Disorder"]["Name"])
        sec_tree= nodo["Disorder"]["HPODisorderAssociationList"]
        if (len(sec_tree)==2):
            tam_sintomas=len(sec_tree["HPODisorderAssociation"])
            i=0
            registro_enfer_sinto_prob=[]
            while (i<tam_sintomas):
                registro_enfer_sinto_prob.append(id)
                registro_enfer_sinto_prob.append(enfermedad)
                registro_enfer_sinto_prob.append(sec_tree["HPODisorderAssociation"][i]["HPO"]["HPOTerm"])
                frecuencia=_clean_name(sec_tree["HPODisorderAssociation"][i]["HPOFrequency"]["Name"])
                registro_enfer_sinto_prob.append(frecuencia)
                lista_enfer_sinto_prob.append (registro_enfer_sinto_prob)
                registro_enfer_sinto_prob=[]
                i=i+1
        id=id+1
    df_enfermedades = pd.DataFrame(lista_enfer_sinto_prob)
    df_enfermedades = df_enfermedades.rename(columns={0:'Id_Enfermedad',1:'Enfermedad', 2:'Sintoma', 3:"Frecuencia"})
    
    return df_enfermedades        

_clean_name: Función privada, que usa load_from_xml, para eliminar carácteres innecesarios en los nombres encontrados en el XML fuente (para enfermedades y frecuencias) 

In [4]:
def _clean_name (cadena):
    
    cadena_str= str(cadena)
    cadena_str=cadena_str[26:]
    cadena_str = cadena_str.replace('}', '',1)
    cadena_str = cadena_str.replace('\'','')
    
    return cadena_str

# Exploratory Data Analysis (EDA) & Feature Engineering functions

eda_data: Función que dado un dataframe de entrada, aplica Exploratory Data Analysis. En concreto: elimina registros duplicados, los nulos, y los registros que contienen síntomas que sólo aparecen menos de 50 veces en nuestra muestra

In [5]:
def eda_data(data): 
    
    data=data.drop_duplicates()
    data=data.dropna()
    vc = data["Sintoma"].value_counts()
    vector=vc[vc < 50].index
    for a in vector:
        indexNames = data [ data["Sintoma"] == a ].index
        for b in indexNames:
            data.drop(b , inplace=True, axis=0)
  
    return data 

selection_data_by_frecuency: Función que dado un dataframe de entrada, se queda con los registros de Enfermedad-Sintoma-Frecuencia que tengan una frecuencia "Muy frecuente", "Frecuente", "Obligatorio" y "Ocasional". Elimina pues los registros con frecuencia Muy poco frecuente y Excluyente.

In [6]:
def selection_data_by_frecuency(data):
    
   
    data=data[(data['Frecuencia']=="Muy frecuente (99-80%)") |
              (data['Frecuencia']=="Frecuente (79-30%)") |
              (data['Frecuencia']=="Obligatorio (100%)") |
              (data['Frecuencia']=="Ocasional (29-5%)")
              
             ]
        
    return data


# Test/Experiment: Recomendation functions

generate_data_matrix_ratings: Función que dado un dataframe de entrada (con todas los registros existentes entre enfermedades, sus síntomas y la frecuencia de aparición), genera una matriz de enfermedades x sintomas, formada por valores únicamente en celdas donde un síntoma concreto (columna) aparezca en la enfermedad. El valor que tendrá dependerá de la frecuencia de aparición de dicho síntoma en la enfermedad (puntos) de tal forma que si es una frecuencia alta la aparición del síntoma en la enfermedad, tendrá más puntuación que una frecuencia más baja. 
Se persigue con esto disponer de una matriz de puntuaciones/ratings donde cruzamos todos los síntomas con todas las enfermedades, con un conjunto de puntuaciones.

In [32]:
def generate_data_matrix_ratings (data, repeticiones):
    
    sintomas=data.iloc[:,1]
    sintomas_sin_repe=sintomas.drop_duplicates()
    sintomas_sin_repe=sintomas_sin_repe.sort_values(ascending
                              = True)
    df_train=pd.DataFrame(columns=sintomas_sin_repe)
  
    data_agrupado = (data.groupby("Enfermedad")
         .agg({"Sintoma": np.array, "Frecuencia": np.array})
         .reset_index()
         )
    
    z=0
    j=0
    while (z<repeticiones):
    
  #  print ("entra")
        i=0
        for a in data_agrupado["Enfermedad"]:
           # print ("Enfermedad: ", a)

            lst = [0] * ((len(sintomas_sin_repe)))
           
            df_train.loc[len(df_train)] = lst
           

           
            pos=0
            for b in data_agrupado["Sintoma"][i]:
                #valor_aleatorio = random.random()
                frecuencia=data_agrupado["Frecuencia"][i][pos]
  
                if (frecuencia=="Muy frecuente (99-80%)"):

                        valor_entero=3
 
                elif (frecuencia=="Frecuente (79-30%)"):

                        valor_entero=2

                         
                elif (frecuencia=="Obligatorio (100%)"):
                    valor_entero=4

                elif (frecuencia=="Ocasional (29-5%)"):
                    valor_entero=1
                    
                df_train[b][j]=valor_entero
               
                pos=pos+1
            j=j+1
            i=i+1
        z=z+1
        print ("VUELTA: ", z)
        print ("Tamaño: ",j)
    return df_train      

Matriz de recomendaciones

In [8]:
def init_collaborative_filtering_user_based (ratings):
    
    #sim_matrix = 1 - sklearn.metrics.pairwise.cosine_distances(ratings)
    sim_matrix= sklearn.metrics.pairwise.cosine_similarity(ratings)
    sintomas_k = sim_matrix.dot(ratings) / np.array([np.abs(sim_matrix).sum(axis=1)]).T
  


    conn = sqlite3.connect('test_database')
    c = conn.cursor()


    df = pd.DataFrame(sintomas_k)
    df=df.transpose()
    df.to_sql('scorings_tfm', conn, if_exists='replace', index = False)
 
   


    

Predicción a partir de matriz de recomendaciones

In [9]:
def predict_collaborative_filtering_user_based(sintoma, elementos):
 
 
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]
    
    
    conn = sqlite3.connect('test_database')
    c = conn.cursor()
    c.execute('''  SELECT * FROM scorings_new
          ''')
    #Predicciones (las recomendaciones!)
    sintomas_k = pd.DataFrame(c.fetchall())  
    sintomas_k=sintomas_k.transpose()
    #print (sintomas_k)
    sintomas_k=sintomas_k.to_numpy()
       
    
    
    #Predicciones (las recomendaciones!)
    
    sintoma_scoring=sintomas_k.argsort()[id_sintoma]
    vector_id_enfermedad_scoring=[]
    for i, enfermedad_id in enumerate(sintoma_scoring[-elementos:]):
        v=[]
        v.append (enfermedad_id)
        v.append (sintomas_k[id_sintoma][enfermedad_id])
        vector_id_enfermedad_scoring.append(v)
        
    vector_id_enfermedad_scoring=pd.DataFrame(vector_id_enfermedad_scoring)
    vector_id_enfermedad_scoring = vector_id_enfermedad_scoring.rename(columns={1:id_sintoma, 0:"index"})
    
    listado_completo=_aparece_y_como (vector_id_enfermedad_scoring,id_sintoma, df_Enfermedades)
    

    

    return listado_completo,sintomas_k, vector_id_enfermedad_scoring

Pearson

In [10]:
def predict_similitud_entre_usuarios_by_pearson (df_train_generated,sintoma,elementos):
    
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]

    
    df_transpuesta=df_train_generated
    correlaciones = df_transpuesta.corr(method='pearson')

    MediasUsers = df_transpuesta.mean(numeric_only=True,skipna=True)
    DesviacionesUsers = df_transpuesta.std(numeric_only=True,skipna=True)
    df_ValoracionesNorm = df_transpuesta[MediasUsers.index].sub(MediasUsers, axis='columns')

    df_ValoracionesNorm = df_ValoracionesNorm.div(DesviacionesUsers, axis='columns')

    Numerador = df_ValoracionesNorm.dot(correlaciones)
    Numerador.sort_values(0,ascending=False)
    Denominador = correlaciones.abs().sum()
    Cociente = Numerador.div(Denominador, axis='columns')

    Valoraciones = Cociente.mul(DesviacionesUsers,axis='columns').add(MediasUsers, axis='columns')

    a=Valoraciones[id_sintoma].sort_values(ascending=False).head(elementos)
    
    
    
    
    
    vector=pd.DataFrame(a)
    vector=vector.reset_index()
    lista=_aparece_y_como (vector,id_sintoma, df_Enfermedades)
    
    
 
    
    return lista,a

Knearest

In [46]:
def _aparece_y_como (scoring_enfermedades,id_Sintoma, df_Enfermedades):
    

    
    j=0
    enfermedades=[]
    while (j<len(scoring_enfermedades)):
        enfermedad=[]
        id_enfermedad=scoring_enfermedades["index"][j]
        scoring=scoring_enfermedades[id_Sintoma][j]
        
        enfermedad.append(id_enfermedad)
       
        enfermedad.append(df_Enfermedades[df_Enfermedades["index"]==id_enfermedad]["Enfermedad"].values[0])
   
        enfermedad.append(scoring)
        lista=df_EnfeySinto_select[df_EnfeySinto_select["Enfermedad"]==
                                   df_Enfermedades.loc[id_enfermedad][1]]
        lista=lista.reset_index()
        sintoma= df_Sintomas.loc[id_Sintoma].Sintoma
        
        i=0
        while i<len(lista):
         
            if lista["Sintoma"][i]==sintoma:
                enfermedad.append(lista["Frecuencia"][i])
            
            i=i+1  
        j=j+1
        enfermedades.append(enfermedad)
    return enfermedades

Con más sintomas el pearson

In [15]:


#common = set(first).intersection(second)
##HACE INTERSECCIÓN DE VARIOS SINTOMAS
def trata_sintomas1 (sintomas,df_transpuesta):
    comunes=[]
    primera_iter=True


    for i in sintomas:
        vector=[]
        lista,vector=predict_similitud_entre_usuarios_by_pearson(df_transpuesta,i,200)
        #listado_completo,users_predictions,ratings_train, ratings_test, sim_matrix, v=predict_collaborative_filtering_ser_based(
    #df_matrix.values, i,10)
        v=vector.reset_index()
        
        sintoma=v["index"]
        sintoma=list(sintoma)
      
        if primera_iter:
            comunes=sintoma
            primera_iter=False

        else:            
            comunes = set(comunes).intersection(sintoma)
        
    return comunes

Con más síntomas el el user based

In [16]:


#common = set(first).intersection(second)
##HACE INTERSECCIÓN DE VARIOS SINTOMAS
def trata_sintomas2 (sintomas,df_transpuesta):
    comunes=[]
    primera_iter=True
    df_matrix=df_transpuesta.transpose()
    
    init_collaborative_filtering_user_based (df_matrix.values)
    
    for i in sintomas:
        vector=[]
        #lista,vector=predict_similitud_entre_usuarios_by_pearson(df_transpuesta,i,20)
        listado_completo,users_predictions, v=predict_collaborative_filtering_user_based(i,200)
        #v=vector.reset_index()
        
        sintoma=v["index"]
        sintoma=list(sintoma)
        #print(sintoma)
      
        if primera_iter:
            comunes=sintoma
            primera_iter=False

        else:            
            comunes = set(comunes).intersection(sintoma)
            
    return comunes


_saca_enfermedades: Función que devuelve un vector de Enfermedades (su literal), dado un vector de ids. Es utilizado únicamente par ver resultados y comparar, ya que es más fácil ver los resultados de los sistemas recomendador con el propio literal de la Enfermedad que con su Id.

In [17]:
def _saca_enfermedades (vector,df_Enfermedades):
    
    enfermedades=[]
    for j in vector:
        enfermedades.append(df_Enfermedades[df_Enfermedades["index"]==j]["Enfermedad"])
        
    return enfermedades

_rename_col: Función que, dado un dataframe con un conjunto de columnas, modifica el nombre de dichas columnas para poder trabajar con su Id, en lugar de literales.




In [18]:
def _rename_col(df):
    
    df.rename(columns={'Sintoma':'Enfermedades'},
               inplace=True)
    columnas=len(df.columns)
    i=0
    while (i<columnas):
        df = df.rename(columns={df.columns[i]:i})
        i=i+1
        
        
    return df

# Notebook Main

## Common Tasks



Vayamos por pasos:

Se comienza con la lectura del fichero XML fuente de Orphadata, llamando a la función definida anteriormente

In [19]:
df_EnfeySinto=load_from_xml("../data/01_raw/enfermedades.xml")


A modo visual, vemos qué hemos obtenido de esta lectura, tanto a nivel de Enfermedades, como síntomas y frecuencias de aparición

In [20]:
print ("Datos de Lectura del XML Fuente Orphadata")
print ("Enfermedades: ", df_EnfeySinto["Enfermedad"].nunique())
print ("Sintomas: ", df_EnfeySinto["Sintoma"].nunique())
print ("Frecuencias: ", df_EnfeySinto["Frecuencia"].nunique())  

Datos de Lectura del XML Fuente Orphadata
Enfermedades:  4262
Sintomas:  8303
Frecuencias:  6


In [21]:
%%time
df_EnfeySinto_cleaned=eda_data(df_EnfeySinto)


CPU times: user 5min 15s, sys: 714 ms, total: 5min 16s
Wall time: 5min 16s


In [22]:
print ("Después de EDA")
print ("Enfermedades: ", df_EnfeySinto_cleaned["Enfermedad"].nunique())
print ("Sintomas: ", df_EnfeySinto_cleaned["Sintoma"].nunique())
print ("Frecuencias: ", df_EnfeySinto_cleaned["Frecuencia"].nunique())  

Después de EDA
Enfermedades:  4187
Sintomas:  483
Frecuencias:  6


In [23]:
df_EnfeySinto_select=selection_data_by_frecuency(df_EnfeySinto_cleaned)


In [24]:
df_EnfeySinto_select

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
0,0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,0,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,0,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,0,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,0,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...,...
112669,4262,Artrogriposis múltiple congénita miogénica aut...,Depressed nasal bridge,Ocasional (29-5%)
112671,4262,Artrogriposis múltiple congénita miogénica aut...,Postnatal growth retardation,Ocasional (29-5%)
112672,4262,Artrogriposis múltiple congénita miogénica aut...,Gastrostomy tube feeding in infancy,Ocasional (29-5%)
112679,4263,Nevo de cabello lanoso,Fine hair,Muy frecuente (99-80%)


In [25]:
  print ("Después de Selection")
  print ("Enfermedades: ", df_EnfeySinto_select["Enfermedad"].nunique())
  print ("Sintomas: ", df_EnfeySinto_select["Sintoma"].nunique())
  print ("Frecuencias: ", df_EnfeySinto_select["Frecuencia"].nunique())  


Después de Selection
Enfermedades:  4173
Sintomas:  483
Frecuencias:  4


Ya disponemos de la información preparada para avanzar con los experimentos. Se comienza primero con el experimento que se planteo de forma inicial

## Recomendator

### Datos Ratings Matrix

In [26]:
df_EnfeySinto_select_no_id=df_EnfeySinto_select.drop("Id_Enfermedad", axis=1)
df_EnfeySinto_select_no_id

,Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...
112669,Artrogriposis múltiple congénita miogénica aut...,Depressed nasal bridge,Ocasional (29-5%)
112671,Artrogriposis múltiple congénita miogénica aut...,Postnatal growth retardation,Ocasional (29-5%)
112672,Artrogriposis múltiple congénita miogénica aut...,Gastrostomy tube feeding in infancy,Ocasional (29-5%)
112679,Nevo de cabello lanoso,Fine hair,Muy frecuente (99-80%)


In [27]:
df_Enfermedades=df_EnfeySinto_select_no_id.groupby (["Enfermedad"]).count().reset_index()
df_Enfermedades=df_Enfermedades.drop(["Sintoma","Frecuencia"], axis=1)
df_Enfermedades=df_Enfermedades.reset_index()
df_Enfermedades

,index,Enfermedad
0,0,AICA ribosiduria
1,1,ALG1-CDG
2,2,ALG11-CDG
3,3,ALG12-CDG
4,4,ALG13-CDG
...,...,...
4168,4168,Xeroderma pigmentoso
4169,4169,Xeroderma pigmentoso tipo variante
4170,4170,Zigomicosis
4171,4171,síndrome de talla baja-sordera-disfunción neut...


In [28]:
df_Sintomas=df_EnfeySinto_select_no_id.groupby (["Sintoma"]).count().reset_index()
df_Sintomas=df_Sintomas.drop(["Enfermedad","Frecuencia"], axis=1)
df_Sintomas

,Sintoma
0,Abdominal distention
1,Abdominal pain
2,Abnormal bleeding
3,Abnormal blistering of the skin
4,Abnormal cardiac septum morphology
...,...
478,Weight loss
479,Wide intermamillary distance
480,Wide mouth
481,Wide nasal bridge


In [33]:
%%time
df_train_generated=generate_data_matrix_ratings (df_EnfeySinto_select_no_id, 1)

VUELTA:  1
Tamaño:  4173
CPU times: user 7.22 s, sys: 2.11 s, total: 9.33 s
Wall time: 9.36 s


In [34]:
df_train_generated

Sintoma,Abdominal distention,Abdominal pain,Abnormal bleeding,Abnormal blistering of the skin,Abnormal cardiac septum morphology,Abnormal cerebral white matter morphology,Abnormal dermatoglyphics,Abnormal enzyme/coenzyme activity,Abnormal facial shape,Abnormal fingernail morphology,...,Visual impairment,Visual loss,Vomiting,Waddling gait,Webbed neck,Weight loss,Wide intermamillary distance,Wide mouth,Wide nasal bridge,Wide nose
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4170,0,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
df_train_generated=_rename_col(df_train_generated)
df_train_generated



Sintoma,0,1,2,3,4,5,6,7,8,9,...,473,474,475,476,477,478,479,480,481,482
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4170,0,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df_matrix=df_train_generated.transpose()
df_matrix

,0,1,2,3,4,5,6,7,8,9,...,4163,4164,4165,4166,4167,4168,4169,4170,4171,4172
Sintoma,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
480,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Collaborative Filtering: User based

In [47]:
init_collaborative_filtering_user_based (df_matrix.values)

In [48]:
listado_completo,users_predictions,v=predict_collaborative_filtering_user_based("Cough",10)

In [49]:
listado_completo

[[2207, 'Polimiositis', 0.63671479525537, 'Muy frecuente (99-80%)'],
 [1423, 'Gripe aviar', 0.6382757844473661, 'Muy frecuente (99-80%)'],
 [2480,
  'Síndrome antisintetasa',
  0.6524342127715518,
  'Muy frecuente (99-80%)'],
 [1689,
  'Linfangioleiomiomatosis',
  0.6527724664337454,
  'Muy frecuente (99-80%)'],
 [4029, 'Tifus de las malezas', 0.6612850581965117, 'Muy frecuente (99-80%)'],
 [1207,
  'Enfermedad por anticuerpos anti-membrana basal glomerular',
  0.6746039663251943,
  'Muy frecuente (99-80%)'],
 [1189,
  'Enfermedad de los legionarios',
  0.7470164299505365,
  'Muy frecuente (99-80%)'],
 [1972,
  'Neumonía necrotizante estafilocócica',
  0.7474481267629826,
  'Muy frecuente (99-80%)'],
 [2715,
  'Síndrome de Hughes-Stovin',
  0.772552060870626,
  'Muy frecuente (99-80%)'],
 [1421,
  'Granulomatosis con poliangeítis',
  0.8535007869582255,
  'Frecuente (79-30%)']]

## Similitud entre Usuarios: Correlación de Pearson

SOLO UN SINTOMA

In [54]:
listado_completo, vector_enfermedades_scoring=predict_similitud_entre_usuarios_by_pearson(
    df_train_generated,"Cough", 10)


In [55]:
listado_completo

[[2715,
  'Síndrome de Hughes-Stovin',
  0.6900752891765587,
  'Muy frecuente (99-80%)'],
 [1972,
  'Neumonía necrotizante estafilocócica',
  0.6590546575580434,
  'Muy frecuente (99-80%)'],
 [1421,
  'Granulomatosis con poliangeítis',
  0.646718690748427,
  'Frecuente (79-30%)'],
 [1207,
  'Enfermedad por anticuerpos anti-membrana basal glomerular',
  0.6285446552044739,
  'Muy frecuente (99-80%)'],
 [345, 'Blastoma pulmonar', 0.6040833380603052, 'Muy frecuente (99-80%)'],
 [1189,
  'Enfermedad de los legionarios',
  0.5865656810364697,
  'Muy frecuente (99-80%)'],
 [1689,
  'Linfangioleiomiomatosis',
  0.5565176790671993,
  'Muy frecuente (99-80%)'],
 [215, 'Aspergilosis', 0.5408371321960674, 'Muy frecuente (99-80%)'],
 [4155,
  'Vasculitis urticarial hipocomplementémica',
  0.5121208314395282,
  'Frecuente (79-30%)'],
 [2480,
  'Síndrome antisintetasa',
  0.5036944902150464,
  'Muy frecuente (99-80%)']]

In [56]:
vector_enfermedades_scoring

2715    0.690075
1972    0.659055
1421    0.646719
1207    0.628545
345     0.604083
1189    0.586566
1689    0.556518
215     0.540837
4155    0.512121
2480    0.503694
Name: 115, dtype: float64

In [57]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Brucelosis")]

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
94850,3797,Brucelosis,Hyperhidrosis,Frecuente (79-30%)
94851,3797,Brucelosis,Arthritis,Frecuente (79-30%)
94852,3797,Brucelosis,Splenomegaly,Frecuente (79-30%)
94853,3797,Brucelosis,Weight loss,Frecuente (79-30%)
94855,3797,Brucelosis,Anemia,Frecuente (79-30%)
94856,3797,Brucelosis,Fever,Frecuente (79-30%)
94857,3797,Brucelosis,Nausea,Frecuente (79-30%)
94858,3797,Brucelosis,Anorexia,Frecuente (79-30%)
94859,3797,Brucelosis,Hepatomegaly,Frecuente (79-30%)
94860,3797,Brucelosis,Arthralgia,Frecuente (79-30%)


 CON MULTI-SINTOMA

In [58]:
SINTOMAS_EJEMPLO = ['Cough','Dysphagia']

In [59]:
sintomas_comunes1=trata_sintomas1 (SINTOMAS_EJEMPLO,df_train_generated)
sintomas_comunes1

{11, 394, 399, 1090, 1306, 1360, 1369, 1422, 1956, 2005, 2211, 2529, 2940}

In [60]:
print(_saca_enfermedades (sintomas_comunes1,df_Enfermedades))

[2529    Síndrome de Alport ligado al cromosoma X-leiom...
Name: Enfermedad, dtype: object, 1090    Enfermedad de Gaucher tipo 2
Name: Enfermedad, dtype: object, 2211    Poliomielitis
Name: Enfermedad, dtype: object, 1956    Necrólisis epidérmica tóxica
Name: Enfermedad, dtype: object, 394    Carcinoma anaplásico de tiroides
Name: Enfermedad, dtype: object, 11    Acalasia idiopática
Name: Enfermedad, dtype: object, 1422    Granulomatosis eosinofílica con poliangeítis
Name: Enfermedad, dtype: object, 399    Carcinoma de esófago
Name: Enfermedad, dtype: object, 1360    Fiebre de Lassa
Name: Enfermedad, dtype: object, 2005    Neuropatía sensitivo-motora hereditaria, tipo ...
Name: Enfermedad, dtype: object, 1369    Fiebre hemorrágica del virus Ébola
Name: Enfermedad, dtype: object, 1306    Esclerosis sistémica cutánea difusa
Name: Enfermedad, dtype: object, 2940    Síndrome de Stevens-Johnson
Name: Enfermedad, dtype: object]


In [61]:
sintomas_comunes2=trata_sintomas2 (SINTOMAS_EJEMPLO,df_train_generated)
sintomas_comunes2

{399,
 493,
 645,
 1088,
 1090,
 1155,
 1156,
 1306,
 1369,
 1422,
 1956,
 2529,
 2786,
 2940,
 2997,
 3935}

In [62]:
print(_saca_enfermedades (sintomas_comunes2,df_Enfermedades))

[1088    Enfermedad de Gaucher
Name: Enfermedad, dtype: object, 2529    Síndrome de Alport ligado al cromosoma X-leiom...
Name: Enfermedad, dtype: object, 2786    Síndrome de Lynch
Name: Enfermedad, dtype: object, 1155    Enfermedad de Whipple
Name: Enfermedad, dtype: object, 1156    Enfermedad de Wilson
Name: Enfermedad, dtype: object, 645    Dermatomiositis juvenil
Name: Enfermedad, dtype: object, 1956    Necrólisis epidérmica tóxica
Name: Enfermedad, dtype: object, 1090    Enfermedad de Gaucher tipo 2
Name: Enfermedad, dtype: object, 493    Cáncer colorrectal familiar tipo X
Name: Enfermedad, dtype: object, 1422    Granulomatosis eosinofílica con poliangeítis
Name: Enfermedad, dtype: object, 399    Carcinoma de esófago
Name: Enfermedad, dtype: object, 2997    Síndrome de Williams
Name: Enfermedad, dtype: object, 1369    Fiebre hemorrágica del virus Ébola
Name: Enfermedad, dtype: object, 1306    Esclerosis sistémica cutánea difusa
Name: Enfermedad, dtype: object, 2940    Síndrome de 

In [63]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Granulomatosis eosinofílica con poliangeítis") &
                     ((df_EnfeySinto_select["Sintoma"]=="Cough") | (df_EnfeySinto_select["Sintoma"]=="Weight loss") 
                     | (df_EnfeySinto_select["Sintoma"]=="Dysphagia") | (df_EnfeySinto_select["Sintoma"]=="Gastroesophageal reflux"))]

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
8771,290,Granulomatosis eosinofílica con poliangeítis,Weight loss,Muy frecuente (99-80%)
8775,290,Granulomatosis eosinofílica con poliangeítis,Dysphagia,Frecuente (79-30%)
8777,290,Granulomatosis eosinofílica con poliangeítis,Gastroesophageal reflux,Ocasional (29-5%)
8799,290,Granulomatosis eosinofílica con poliangeítis,Cough,Ocasional (29-5%)
